In [133]:
from easydict import EasyDict as edict
import json, os
import torch

import pandas as pd
import numpy as np

from scipy.special import expit

import matplotlib.pyplot as plt

In [102]:
import sys
sys.path.append('/home/users/jsoelter/Code/Chexpert/')
from model.classifier import Classifier
from data.dataset import ImageDataset  

In [103]:
data_path = '/work/projects/covid19_dv/raw_data/heavy_datasets/chexpert_stanford/'

In [18]:
original_csv = pd.read_csv(os.path.join(data_path, 'CheXpert-v1.0/valid.csv'))

# add the correct location of the images
modified_csv = original_csv.copy()
modified_csv.Path = modified_csv.Path.apply(lambda x: os.path.join(data_path, x))

#### Write test data file to disk

In [106]:
prediction_csv = '/home/users/jsoelter/chexpert_eval/test01.csv'

dirname = os.path.dirname(prediction_csv)
if not os.path.exists(dirname):
    os.makedirs(dirname)

In [107]:
modified_csv.to_csv(prediction_csv, index=False)

#### Load model

In [108]:
args = {
    'config': '/home/users/jsoelter/Code/Chexpert/config/example.json',
    'model_weight': '/home/users/jsoelter/Code/Chexpert/config/pre_train.pth',       # "Path to the trained models"
    'in_csv_path': prediction_csv,   # "Path to the input image path in csv"
    'num_workers': 8,                # "workers for each data loader"
}

In [109]:
device = 'cuda:0'

In [116]:
# open config
with open(args['config']) as f:
    cfg = edict(json.load(f))

# create model
model = Classifier(cfg)
_ = model.to(device)

# load pretrained weights/parameter
ckpt = torch.load(args['model_weight'], map_location='cpu')
model.load_state_dict(ckpt)

<All keys matched successfully>

### Create Dataloader

In [117]:
dataset = ImageDataset(args['in_csv_path'], cfg, mode='test')
dataloader_test = torch.utils.data.DataLoader(
    dataset,
    batch_size=cfg.dev_batch_size, 
    num_workers=args['num_workers'],
    drop_last=False, 
    shuffle=False
)

In [124]:
prediction_pathology = [
    'Cardiomegaly',
    'Edema',
    'Consolidation',
    'Atelectasis',
    'Pleural Effusion'
]

In [142]:
model.eval()

predictions = []
for image_batch, path in dataloader_test:
    with torch.no_grad():
        batch_prediction, _ = model(image_batch.to(device))
    batch_prediction = np.hstack([p.cpu().numpy() for p in batch_prediction])
    predictions.append(batch_prediction)
predictions = np.vstack(predictions)

In [143]:
probabilities = expit(predictions)
probabilities = pd.DataFrame(probabilities, columns=prediction_pathology)

In [144]:
import sklearn.metrics as skm

In [145]:
col = 'Atelectasis'

skm.roc_auc_score(modified_csv[col], probabilities[col])

0.8949675324675325